# Comparing channels

- R: sequential
- G: complementary
- B: not match at all

In [17]:
import ppscore
import pandas
import os
import numpy
from scipy.stats import spearmanr
from IPython.core.display import display, HTML
from ete3 import Tree

/tmp/ipykernel_222316/499083497.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [18]:
tree_path = "../../../data/trees"
channels = ("full", "gray_r", "gray_g", "gray_b", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin"]

In [19]:
def read_and_compare(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(numpy.sqrt(numpy.sum((control_df.values - df.values)**2)), 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Euclidean Distance from distance matrices

In [20]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_compare(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.9085,1.8686,0.0,0.8937,1.8103,1.8433,1.9548,1.1396
orthologs_myoglobin,0.9196,1.9899,0.0,0.9044,1.8112,1.9487,2.2140,1.1520
orthologs_neuroglobin,1.8043,5.8999,0.0,1.1514,5.8504,5.8717,5.9314,1.7182
orthologs_cytoglobin,6.3417,6.5979,0.0,3.1095,6.6067,6.6341,6.5003,3.2222


In [21]:
def read_and_linear_correlate(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(spearmanr(control_df.values.flatten(), df.values.flatten())[0], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Linear Correlation from distance matrices

In [22]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_linear_correlate(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.9999,0.9978,1.0,0.9998,0.9976,0.9984,0.9919,NaN
orthologs_myoglobin,0.9999,0.9972,1.0,0.9998,0.9950,0.9977,0.9922,NaN
orthologs_neuroglobin,0.9999,0.9858,1.0,0.9992,0.9868,0.9935,0.9802,NaN
orthologs_cytoglobin,0.7775,0.4569,1.0,0.8623,0.4521,0.4544,0.4514,NaN


In [23]:
def read_and_correlate(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(ppscore.score(
                    pandas.DataFrame({"x": control_df.values.flatten(), "y": df.values.flatten()}), "x", "y")["ppscore"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Predictive Power Score from distance matrices

In [24]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_correlate(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.9953,0.9792,0.9968,0.9944,0.9801,0.9824,0.9484,0.0
orthologs_myoglobin,0.9954,0.9699,0.9968,0.9925,0.9710,0.9715,0.9210,0.0
orthologs_neuroglobin,0.9876,0.9681,0.9964,0.9897,0.9649,0.9659,0.9622,0.0
orthologs_cytoglobin,0.8524,0.7308,0.9913,0.8584,0.7430,0.7417,0.7133,0.0


In [25]:
def read_and_tree_compare(dataset, channel="full", control="Control with Clustal Omega"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_tree = Tree(f"{path}/{control}.nw", format=1)
    for file in os.listdir(path):
        if file.endswith(".nw"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                tree = Tree(f"{path}/{file}", format=1)
                result = control_tree.compare(tree, unrooted=True)
                result_dict[basename] = round(result["norm_rf"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Robinson-foulds distance from generated Trees

In [26]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.1667,1.0
orthologs_myoglobin,0.0000,0.0,0.0000,0.0,0.0833,0.0833,0.1667,1.0
orthologs_neuroglobin,0.0000,0.0,0.0833,0.0,0.0833,0.0833,0.0833,1.0
orthologs_cytoglobin,0.6667,0.5,0.5833,0.0,0.5833,0.5833,0.5833,1.0


In [27]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel, "Global with Needleman-Wunsch"))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.0,0.00,0.0000,0.0000,0.0000,0.0000,0.1667,1.0
orthologs_myoglobin,0.0,0.00,0.0000,0.0000,0.0833,0.0833,0.1667,1.0
orthologs_neuroglobin,0.0,0.00,0.0833,0.0000,0.0833,0.0833,0.0833,1.0
orthologs_cytoglobin,0.0,0.25,0.6667,0.6667,0.6667,0.6667,0.6667,1.0


In [28]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel, "Local with Smith–Waterman"))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.00,0.0,0.0000,0.0,0.0000,0.0000,0.1667,1.0
orthologs_myoglobin,0.00,0.0,0.0000,0.0,0.0833,0.0833,0.1667,1.0
orthologs_neuroglobin,0.00,0.0,0.0833,0.0,0.0833,0.0833,0.0833,1.0
orthologs_cytoglobin,0.25,0.0,0.5000,0.5,0.5000,0.5000,0.5000,1.0


In [29]:
def read_and_tree_compare_branches(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_tree = Tree(f"{path}/Control with Clustal Omega.nw", format=1)
    for file in os.listdir(path):
        if file.endswith(".nw"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                tree = Tree(f"{path}/{file}", format=1)
                result = control_tree.compare(tree, unrooted=True)
                result_dict[basename] = 1.0 - round(result["source_edges_in_ref"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Compatibility branch score from generated Trees

In [30]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare_branches(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0769,0.4615
orthologs_myoglobin,0.0000,0.0000,0.0000,0.0,0.0385,0.0385,0.0769,0.4615
orthologs_neuroglobin,0.0000,0.0000,0.0385,0.0,0.0385,0.0385,0.0385,0.4615
orthologs_cytoglobin,0.3077,0.2308,0.2692,0.0,0.2692,0.2692,0.2692,0.4615


In [31]:
def read_and_compare_channels(dataset):
    result_dict = {}
    dfs = {}
    for channel in channels:
        result_dict[channel] = {}
        path = f"{tree_path}/{channel}/{dataset}"
        dfs[channel] = pandas.read_csv(f"{path}/Structural Similarity Index Measure.csv", index_col=0)
    for channel in channels:
        control_df = dfs[channel]
        for c, df in dfs.items():
            result_dict[channel][c] = round(numpy.sqrt(numpy.sum((control_df.values - df.values)**2)), 4)
    result_df = pandas.DataFrame.from_dict(result_dict, orient='index')
    result_df.index.name = "dataset"
    return result_df.style.background_gradient(axis=None, vmin=result_df.min().min(), vmax=result_df.max().max(), cmap="YlGnBu")

## Features Euclidean distance from distance matrices

In [32]:
for dataset in datasets:
    try:
        correlations = read_and_compare_channels(dataset)
    except FileNotFoundError:
        continue
    else:
        display(HTML(f"<center><h3>{dataset}</h3></center>"))
        display(correlations)        

,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,0.092800,0.075600,0.164500,3.006000
gray_r,0.092800,0.000000,0.042600,0.255600,2.948500
gray_g,0.075600,0.042600,0.000000,0.238600,2.981700
gray_b,0.164500,0.255600,0.238600,0.000000,3.087100
gray_mean,3.006000,2.948500,2.981700,3.087100,0.000000


,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,0.208400,0.085800,0.258700,3.140400
gray_r,0.208400,0.000000,0.179700,0.462700,2.961600
gray_g,0.085800,0.179700,0.000000,0.326700,3.098400
gray_b,0.258700,0.462700,0.326700,0.000000,3.362200
gray_mean,3.140400,2.961600,3.098400,3.362200,0.000000


,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,0.167900,0.137100,0.300100,7.459400
gray_r,0.167900,0.000000,0.041200,0.464600,7.403500
gray_g,0.137100,0.041200,0.000000,0.435900,7.427000
gray_b,0.300100,0.464600,0.435900,0.000000,7.500000
gray_mean,7.459400,7.403500,7.427000,7.500000,0.000000


,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,0.248700,0.245600,0.306300,9.372200
gray_r,0.248700,0.000000,0.041200,0.441700,9.377900
gray_g,0.245600,0.041200,0.000000,0.442200,9.405400
gray_b,0.306300,0.441700,0.442200,0.000000,9.276100
gray_mean,9.372200,9.377900,9.405400,9.276100,0.000000
